In [1]:
import pandas as pd
import numpy as np
import sys
import os
import random
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-07-06 21:50:48.416038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 21:50:48.485158: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-06 21:50:48.487163: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-06 21:50:48.487172: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [4]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [5]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


#### Model training and validation

In [6]:
def ffnn(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
    model = tf.keras.Sequential() 
    model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
#     model.add(Dropout(rate=dropout))
    model.add(Dense(128, activation='relu', name='Hidden-Layer1'))
#     model.add(Dropout(rate=dropout))
    model.add(Dense(32, activation='relu', name='Hidden-Layer2'))
#     model.add(Dropout(rate=dropout))
    model.add(Dense(1, activation='linear', name='Output-Layer')) 

    model.compile(optimizer=optimizer, # default='rmsprop', an algorithm to be used in backpropagation
                  loss=tf.keras.losses.MeanSquaredError(),
                 )
    model.fit(X_train, 
          y_train,
          batch_size=32,
          epochs=epochs, # default=1, Number of epochs to train the model
          callbacks=None,
          validation_split=0.2, 
         )
    return model

In [7]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

#### delete from here

In [8]:
# idx = 0
# device_indices = final_df[final_df.latitude==latitudes[idx]].index
# device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
# assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
# test_df = final_df.loc[device_indices]
# test_df.head()

In [9]:
# train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
# assert(len(train_df.longitude.unique()) == len(longitudes)-1)
# assert len(final_df) == len(test_df) + len(train_df)
# train_df.head()

In [10]:
# X_train = train_df.iloc[:, 0:-1]
# y_train = train_df.iloc[:, -1]
# X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)

# X_test = test_df.iloc[:, 0:-1]
# y_test = test_df.iloc[:, -1]
# X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
# X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [11]:
# X_train[0]

In [12]:
# X_train_scaled = scale_data(X_train)
# X_train_scaled

In [13]:
# model = ffnn(X_train, y_train, 500, 'RMSProp')

In [14]:
# y_pred = model.predict(X_test) 
# y_pred.shape, y_test.reshape(-1, 1).shape

In [15]:
# np.min(y_pred), np.mean(y_pred), np.max(y_pred)

In [16]:
# mean_squared_error(y_test, y_pred)

In [17]:
# y_train_pred = model.predict(X_train)
# mean_squared_error(y_train, y_train_pred)

#### end here

In [18]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = ffnn(X_train, y_train, optimizer='Adam', dropout=0.8)
    y_pred = model.predict(X_test) 
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [ ]:
rmse_list, mape_list = [], []
for i in range(len(latitudes)):
    rmse, mape = cross_validation(final_df, i)
    rmse_list.append(rmse)
    mape_list.append(mape)
    print(f'{device_ids[i]} successful')

Epoch 1/1000


2023-07-06 21:50:50.523981: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-06 21:50:50.524003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-07-06 21:50:50.524006: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-07-06 21:50:50.524087: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-07-06 21:50:50.524099: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.116.04  Release Build  (dvs-builder@U16-I3-A16-3-3)  Thu Apr 27 18:11:06 UTC 2023
GCC version:  gcc 

289/289 [==============================] - 1s 1ms/step - loss: 14429381.0000 - val_loss: 695.1968
Epoch 2/1000
289/289 [==============================] - 0s 934us/step - loss: 440.1732 - val_loss: 630.7939
Epoch 3/1000
289/289 [==============================] - 0s 828us/step - loss: 448.7069 - val_loss: 714.0650
Epoch 4/1000
289/289 [==============================] - 0s 837us/step - loss: 488.5003 - val_loss: 635.2147
Epoch 5/1000
289/289 [==============================] - 0s 710us/step - loss: 472.7919 - val_loss: 844.3975
Epoch 6/1000
289/289 [==============================] - 0s 750us/step - loss: 530.0157 - val_loss: 988.3795
Epoch 7/1000
289/289 [==============================] - 0s 806us/step - loss: 563.4109 - val_loss: 709.6088
Epoch 8/1000
289/289 [==============================] - 0s 752us/step - loss: 611.3159 - val_loss: 692.2584
Epoch 9/1000
289/289 [==============================] - 0s 793us/step - loss: 601.9983 - val_loss: 868.5683
Epoch 10/1000
289/289 [===============

289/289 [==============================] - 0s 743us/step - loss: 27155.5254 - val_loss: 637.7709
Epoch 75/1000
289/289 [==============================] - 0s 798us/step - loss: 21555.9746 - val_loss: 15788.6992
Epoch 76/1000
289/289 [==============================] - 0s 724us/step - loss: 23540.2852 - val_loss: 2550.6472
Epoch 77/1000
289/289 [==============================] - 0s 916us/step - loss: 22780.8262 - val_loss: 31565.1992
Epoch 78/1000
289/289 [==============================] - 0s 747us/step - loss: 19399.4590 - val_loss: 18866.4238
Epoch 79/1000
289/289 [==============================] - 0s 720us/step - loss: 17685.8828 - val_loss: 89440.0781
Epoch 80/1000
289/289 [==============================] - 0s 716us/step - loss: 21029.6035 - val_loss: 6868.3315
Epoch 81/1000
289/289 [==============================] - 0s 777us/step - loss: 19354.2363 - val_loss: 22005.6074
Epoch 82/1000
289/289 [==============================] - 0s 939us/step - loss: 24778.8652 - val_loss: 9825.4619
Ep

289/289 [==============================] - 0s 699us/step - loss: 4607.2720 - val_loss: 6634.4575
Epoch 148/1000
289/289 [==============================] - 0s 756us/step - loss: 5595.3608 - val_loss: 5961.6587
Epoch 149/1000
289/289 [==============================] - 0s 748us/step - loss: 27815.9180 - val_loss: 2171.8884
Epoch 150/1000
289/289 [==============================] - 0s 908us/step - loss: 839.6008 - val_loss: 649.4758
Epoch 151/1000
289/289 [==============================] - 0s 771us/step - loss: 1331.8673 - val_loss: 626.0139
Epoch 152/1000
289/289 [==============================] - 0s 771us/step - loss: 1933.7330 - val_loss: 15161.0801
Epoch 153/1000
289/289 [==============================] - 0s 829us/step - loss: 5215.1392 - val_loss: 759.6946
Epoch 154/1000
289/289 [==============================] - 0s 734us/step - loss: 3638.9631 - val_loss: 25198.9844
Epoch 155/1000
289/289 [==============================] - 0s 725us/step - loss: 5058.0923 - val_loss: 7355.1499
Epoch 15

...

Epoch 239/1000
308/308 [==============================] - 0s 1ms/step - loss: 401.1481 - val_loss: 621.7477
Epoch 240/1000
308/308 [==============================] - 0s 1ms/step - loss: 24859.2852 - val_loss: 1457.0447
Epoch 241/1000
308/308 [==============================] - 0s 973us/step - loss: 895.8212 - val_loss: 1451.6077
Epoch 242/1000
308/308 [==============================] - 0s 936us/step - loss: 891.3326 - val_loss: 1445.4298
Epoch 243/1000
308/308 [==============================] - 0s 925us/step - loss: 886.2418 - val_loss: 1438.4210
Epoch 244/1000
308/308 [==============================] - 0s 978us/step - loss: 880.5101 - val_loss: 1430.5682
Epoch 245/1000
308/308 [==============================] - 0s 888us/step - loss: 874.1097 - val_loss: 1421.8141
Epoch 246/1000
308/308 [==============================] - 0s 901us/step - loss: 867.0261 - val_loss: 1412.1667
Epoch 247/1000
308/308 [==============================] - 0s 920us/step - loss: 859.2723 - val_loss: 1401.6283
Epoc

308/308 [==============================] - 0s 955us/step - loss: 405.1264 - val_loss: 721.2705
Epoch 314/1000
308/308 [==============================] - 0s 987us/step - loss: 403.3050 - val_loss: 716.7036
Epoch 315/1000
308/308 [==============================] - 0s 935us/step - loss: 401.6209 - val_loss: 712.3529
Epoch 316/1000
308/308 [==============================] - 0s 1ms/step - loss: 400.0564 - val_loss: 708.1414
Epoch 317/1000
308/308 [==============================] - 0s 1ms/step - loss: 398.6054 - val_loss: 704.1463
Epoch 318/1000
308/308 [==============================] - 0s 1ms/step - loss: 397.2650 - val_loss: 700.3107
Epoch 319/1000
308/308 [==============================] - 0s 1ms/step - loss: 396.0431 - val_loss: 696.6428
Epoch 320/1000
308/308 [==============================] - 0s 890us/step - loss: 394.9310 - val_loss: 693.2081
Epoch 321/1000
308/308 [==============================] - 0s 992us/step - loss: 393.9159 - val_loss: 689.9711
Epoch 322/1000
308/308 [=========

308/308 [==============================] - 0s 902us/step - loss: 386.3377 - val_loss: 647.8806
Epoch 388/1000
308/308 [==============================] - 0s 927us/step - loss: 386.3386 - val_loss: 647.8703
Epoch 389/1000
308/308 [==============================] - 0s 906us/step - loss: 386.3371 - val_loss: 647.8506
Epoch 390/1000
308/308 [==============================] - 0s 851us/step - loss: 386.3382 - val_loss: 647.8423
Epoch 391/1000
308/308 [==============================] - 0s 896us/step - loss: 386.3382 - val_loss: 647.8008
Epoch 392/1000
308/308 [==============================] - 0s 887us/step - loss: 386.3367 - val_loss: 647.8260
Epoch 393/1000
308/308 [==============================] - 0s 938us/step - loss: 386.3377 - val_loss: 647.8298
Epoch 394/1000
308/308 [==============================] - 0s 912us/step - loss: 386.3388 - val_loss: 647.8204
Epoch 395/1000
308/308 [==============================] - 0s 920us/step - loss: 386.3382 - val_loss: 647.7878
Epoch 396/1000
308/308 [=

308/308 [==============================] - 1s 3ms/step - loss: 386.3386 - val_loss: 647.7328
Epoch 463/1000
308/308 [==============================] - 1s 3ms/step - loss: 386.3387 - val_loss: 647.7565
Epoch 464/1000
308/308 [==============================] - 1s 3ms/step - loss: 386.3384 - val_loss: 647.7122
Epoch 465/1000
308/308 [==============================] - 1s 3ms/step - loss: 386.3379 - val_loss: 647.7210
Epoch 466/1000
308/308 [==============================] - 1s 3ms/step - loss: 386.3384 - val_loss: 647.7333
Epoch 467/1000
308/308 [==============================] - 1s 3ms/step - loss: 386.3383 - val_loss: 647.7828
Epoch 468/1000
308/308 [==============================] - 1s 2ms/step - loss: 386.3382 - val_loss: 647.7970
Epoch 469/1000
308/308 [==============================] - 1s 3ms/step - loss: 386.3382 - val_loss: 647.7615
Epoch 470/1000
308/308 [==============================] - 1s 3ms/step - loss: 386.3386 - val_loss: 647.7338
Epoch 471/1000
308/308 [===================

Epoch 538/1000
308/308 [==============================] - 0s 916us/step - loss: 386.3375 - val_loss: 647.6793
Epoch 539/1000
308/308 [==============================] - 0s 870us/step - loss: 386.3388 - val_loss: 647.7281
Epoch 540/1000
308/308 [==============================] - 0s 917us/step - loss: 386.3381 - val_loss: 647.7426
Epoch 541/1000
308/308 [==============================] - 0s 874us/step - loss: 386.3378 - val_loss: 647.7473
Epoch 542/1000
308/308 [==============================] - 0s 905us/step - loss: 386.3376 - val_loss: 647.7162
Epoch 543/1000
308/308 [==============================] - 0s 869us/step - loss: 386.3385 - val_loss: 647.7086
Epoch 544/1000
308/308 [==============================] - 0s 925us/step - loss: 386.3387 - val_loss: 647.7117
Epoch 545/1000
308/308 [==============================] - 0s 857us/step - loss: 386.3375 - val_loss: 647.7095
Epoch 546/1000
308/308 [==============================] - 0s 901us/step - loss: 386.3379 - val_loss: 647.7032
Epoch 547/

In [ ]:
mean_rmse = np.mean(rmse_list)          
mean_rmse

In [ ]:
rmse_list